In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127710")
exp = Experiment(workspace=ws, name="quick-starts-ws-127710")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)


In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import ScriptRunConfig
from azureml.core import Environment

# Specify parameter sampler
ps = RandomParameterSampling(
    {
       "--C": uniform(0.05, 1.0),
       "--max_iter": choice(50, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###



sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--C', 0.1 , '--max_iter', 1000],
                      compute_target=aml_compute,
                      environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=2)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
#model = best_run.register_model(model_name='scikit-learn', model_path='outputs/model')

joblib.dump(model, 'model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

dataset = Dataset.Tabular.from_delimited_files(data)

#dataset.take(3).to_pandas_dataframe()



In [ ]:
from train import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
    
train_data, test_data = dataset.random_split(percentage=0.8, seed=223)
# Split the dataset into train and test datasets
#x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=0)

#train_data = (x_train, y_train)
label = "y"

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= train_data,
    label_column_name= label,
    n_cross_validations= 5)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

# choose a name for experiment
experiment_name = 'automl-classification-bmarketing-all'

experiment=Experiment(ws, experiment_name)

remote_run = experiment.submit(automl_config, show_output = False)

remote_run

remote_run.wait_for_completion()


{'runId': 'AutoML_f06ce2e6-1b04-4124-bc0b-02ab47f804f6',
 'target': 'local',
 'status': 'Canceled',
 'startTimeUtc': '2020-11-24T13:49:30.138248Z',
 'endTimeUtc': '2020-11-24T14:11:44.520624Z',
 'warnings': [{'source': 'Run',
   'message': 'The run was terminated due to an interruption while being executed.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-classification-bmarketing-all","subscription_id":"55e71b9d-a209-42c0-8818-ca9cc885909c","resource_group":"aml-quickstarts-127710","workspace_name":"quick-starts-ws-127710","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

best_run_customized, fitted_model_customized = remote_run.get_output()
